# Init

In [ ]:
import pandas as pd

In [ ]:
vcfDf=pd.read_csv('/home/jovyan/efs/all_seq/meta_data/All_20170710.f1_byte2_not_00.vcf.gz'
                  ,comment='#',sep='\s+',header=None)

In [ ]:
vcfDf.columns=['CHROM', 'POS' ,  'ID' , 'REF' ,'ALT', 'QUAL' , 'FILTER', 'INFO']

In [ ]:
vcfDf["REF_len"]=vcfDf['REF'].str.len()

chromToPosRsId=vcfDf.sort_values('REF_len').groupby(['CHROM','POS'])['ID'].first()


In [ ]:
mergedDf_rd_reset=pd.read_pickle('mergedDf_rd_reset.pickle')

In [ ]:
altBaseDf=mergedDf_rd_reset[mergedDf_rd_reset.AltBase]

In [ ]:
refBaseDf=mergedDf_rd_reset[~mergedDf_rd_reset.AltBase]

In [ ]:
indexL=['Run_db','Run_digits','Chr','Pos']
refDepth=refBaseDf.set_index(indexL)['ReadDepth']

In [ ]:
altBaseDf['RefRD']=refDepth[altBaseDf.set_index(indexL).index].values

In [ ]:
altBaseDf['RefRD'].fillna(0,inplace=True)

In [ ]:
altBaseDf['Run_str']=altBaseDf['Run_db'].astype(str)+altBaseDf['Run_digits'].astype(str)

### get per sample altenative allele

In [ ]:
altBaseDf_reI=altBaseDf.set_index(['Chr','Pos','base','Run_str'])
altRdFlatDf=altBaseDf_reI['ReadDepth'].unstack().fillna(0)

In [ ]:
RefRdFlatDf=altBaseDf_reI['RefRD'].unstack().fillna(0)

In [ ]:
AD=RefRdFlatDf.astype(int).astype('str')+':'+altRdFlatDf.astype(int).astype(str)

In [ ]:
sampleDf=AD

### generate more info

In [ ]:
qualS=altBaseDf_reI.groupby(['Chr','Pos','base'])['AverageBaseQuality'].median()
qualS.name='QUAL'

refBase=altBaseDf_reI.groupby(['Chr','Pos','base'])['Ref'].first()
refBase.name='REF'

combinedDepth=(RefRdFlatDf+altRdFlatDf).sum(axis=1)

poolAF=RefRdFlatDf.sum(axis=1)/(RefRdFlatDf.sum(axis=1)+altRdFlatDf.sum(axis=1))#.fillna(0)
infoS="DP="+combinedDepth.astype(int).astype(str)+';AF='+poolAF.astype(str).str[:6]
infoS.name='INFO'

# Params

In [ ]:
thresholdForVars=0.05
thresholdForDepth=10

In [ ]:
FilterS=((combinedDepth>thresholdForDepth)
         &(poolAF>thresholdForVars)).map({True:'PASS',
                                              False:'FAIL'})
FilterS.name='FILTER'

In [ ]:
mergedDf=pd.concat([qualS,refBase,FilterS,infoS,combinedDepth,sampleDf],axis=1)#.reset_index()

In [ ]:
mergedDf.index.names=['#CHROM','POS','ALT']

In [ ]:
mergedDf_reI=mergedDf.reset_index()

In [ ]:
mergedDf_reI['ID']='-'

In [ ]:
mergedDf_reI['FORMAT']='AD'

In [ ]:
mandatoryCols=['#CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT']
mergedDf_reordered=mergedDf_reI[mandatoryCols#
                                +sampleDf.columns.tolist()
                               ]

In [ ]:
mergedDf_reordered.to_csv('output_file.vcf',sep='\t',index=None)